In [1]:

import torch
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


torch: 2.8.0+cu128 cuda: 12.8
CUDA available: True
Device: NVIDIA GeForce RTX 3080
